In [7]:
# Take all JSON from Blob Container and upload to Azure Search

In [8]:
import globals

import os
import pickle

import json
import requests
from pprint import pprint

from azure.storage.blob import BlockBlobService

from joblib import Parallel, delayed

In [9]:
def processLocalFile(file_name):
    json_content = {}
    try:
        with open(file_name, 'r') as json_file:
            json_content = json.loads(json_file.read())
        docID = json_content["paper_id"]
        title = json_content["metadata"]["title"]

        body = {"documents": []}
        
        abstractContent = ''
        id_counter = 1
        if "abstract" in json_content:
            for c in json_content["abstract"]:
                abstractContent += c["text"] + ' '
                body["documents"].append({
                      "language": "en",
                      "id": str(id_counter),
                      "text": c["text"]
                    })
                id_counter += 1

        abstractContent = abstractContent.strip()

        body = ''
        if "body_text" in json_content:
            for c in json_content["body_text"]:
                body += c["text"] + ' '
            body = body.strip()

        contributors = []
        for c in json_content["metadata"]["authors"]:
            midInitial = ''
            for mi in c["middle"]:
                midInitial += mi + ' '
            if len(((c["first"] + ' ' + midInitial + c["last"]).strip())) > 2:
                contributors.append((c["first"] + ' ' + midInitial + c["last"]).strip()) 

        return {"@search.action": "mergeOrUpload", "docID": docID, "title":title, "abstractContent": abstractContent, "body": body, "contributors": contributors}


    except Exception as ex:
        print (blob_name, " - Error:", str(ex))
        return "Error"



In [10]:
with open(os.path.join(globals.files_dir, 'new_files.pkl'), 'rb') as input:
    new_files = pickle.load(input)

In [11]:
print (str(len(new_files)), 'to upload...')

1941 to upload...


In [12]:
documents = {"value": []}
for json_file in new_files:
    # print (json_file[json_file.rindex('/')+1:].replace('.json', '').replace('.xml', ''))
    documents["value"].append(processLocalFile(json_file))
    if len(documents["value"]) == 100:
        print ("Applying", str(len(documents["value"])), "docs...")
        url = globals.endpoint + "indexes/" + globals.indexName + "/docs/index" + globals.api_version
        response  = requests.post(url, headers=globals.headers, json=documents)
        documents = {"value": []}
if len(documents["value"]) > 0:
    print ("Applying", str(len(documents["value"])), "docs...")
    url = globals.endpoint + "indexes/" + globals.indexName + "/docs/index" + globals.api_version
    response  = requests.post(url, headers=globals.headers, json=documents)


Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 100 docs...
Applying 41 docs...
